# Web scraping using BeautifulSoup (alongwith requests + etree)

In [ ]:
# uncomment to install
# !pip install bs4
# !pip install selenium
# !pip install webdriver-manager

In [ ]:
import requests
example = requests.get("http://www.fsf.org/")
html = example.text

from bs4 import BeautifulSoup
soup = BeautifulSoup(html)
print(soup.prettify())

In [ ]:
from lxml import etree

soup = BeautifulSoup(example.content, "html.parser")
e = etree.HTML(str(soup))

for i in e.xpath('//ul[@class="initiatives-list"]/li/a/div/p'):
    print(i.text)

# Web scraping using selenium

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [ ]:
options = Options()
# options.add_argument("--headless")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

driver.get('https://maps.google.com')

search_term = 'NUST Islamabad'
driver.find_element_by_xpath('//*[@id="searchboxinput"]').send_keys(search_term + Keys.RETURN)
time.sleep(7)

In [ ]:
data = driver.find_elements_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[7]/div')

name = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/h1/span[1]').text
rating = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span/span/span').text
reviews = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/span[1]/span/span/span[2]/span[1]/button').text
category = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[2]/span[1]/span[1]/button').text

location = ""
plus_code = ""
website = ""
phone_no = ""
health_safety = ""
located_in = ""

try:
    timings = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[7]/div[3]/div[2]').get_attribute("aria-label")
    if timings == None:                     
        timings = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[7]/div[2]/div[2]').get_attribute("aria-label")
except NoSuchElementException:
    try:
        timings = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[7]/div[2]/div[2]').get_attribute("aria-label")
    except:
        timings = 'Unavailable'


for val in data:
    text = val.text
    if text == data[0].text:
        location = text
        continue
    if text[4] == '+':
        plus_code = text
    elif text[0] == '+':
        phone_no = text
    elif text[:6] == 'Health':
        health_safety = text[17:]
    elif text[:7] == 'Located':
        located_in = text[12:]
    else:
        continue

print(f'Name: {name}\nRating: {rating}\nReviews: {reviews}\nCategoty: {category}\nLocation: {location}\nLocated In: {located_in}\nPhone no: {phone_no}\nTimings: {timings}\nHealth safety: {health_safety}\nPlus code: {plus_code}\n URL: {driver.current_url}')

In [ ]:
driver.close()

# Scraping Daraz.pk

In [ ]:
ratings = [3, 4]
pricing = [(300,500), (1000,1500)]

query = input('Enter search term: ').replace(' ', '%20')
print(f'Modified search term: {query}')

In [ ]:
options = Options()
# options.add_argument("--headless")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

names = []
mall = []
prce = []
star = []
revw = []
for i in pricing:
    for j in ratings:
        driver.get(f'https://www.daraz.pk/catalog/?from=input&q={query}&price={i[0]}-{i[1]}&rating={j}')

        for product in driver.find_elements_by_class_name('c2prKC'):  
            name = product.find_element_by_class_name('c16H9d').text
            
            try:
                price = int(product.find_element_by_class_name('c3gUW0').text[3:].replace(',',''))
            except ValueError:
                price = 0

            prce.append(price)
            names.append(name)
driver.close()
            
for i in range(len(names)):
    print(f'Name: {names[i]}, Price: {prce[i]}')

In [ ]:
df = pd.DataFrame({'Name':names, 'Price':prce})
df

In [ ]:
df['Price'].min()

In [ ]:
df['Price'].max()

# Scraping twitter

In [ ]:
url = 'https://twitter.com/search?q=%23PakvsNz&src=trend_click&vertical=trends'

options = Options()
#options.add_argument("--headless")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

driver.get(url)

In [ ]:
driver.find_element_by_xpath('/html/body/div/div/div/div[2]/main/div/div/div/div/div/div[2]/div/div/section/div/div/div[4]').text

In [ ]:
driver.find_element_by_xpath('/html/body/div/div/div/div[2]/main/div/div/div/div/div/div[1]/div[1]/div/div/div/div/div[2]/div[2]/div/div/div/form/div[1]/div/div/label/div[2]/div/input').send_keys('Imran khan' + Keys.RETURN)
time.sleep(5)
driver.find_element_by_xpath('/html/body/div/div/div/div[2]/main/div/div/div/div/div/div[2]/div/div/section/div/div/div[3]/div/div/div/div[2]/div[1]/div[1]/a/div/div[1]/div[1]/span/span').click()